In [133]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [134]:
# Загружаем данные
df = pd.read_csv('data/wineQualityReds.csv')
df.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1599 non-null   int64  
 1   fixed.acidity         1599 non-null   float64
 2   volatile.acidity      1599 non-null   float64
 3   citric.acid           1599 non-null   float64
 4   residual.sugar        1599 non-null   float64
 5   chlorides             1599 non-null   float64
 6   free.sulfur.dioxide   1599 non-null   float64
 7   total.sulfur.dioxide  1599 non-null   float64
 8   density               1599 non-null   float64
 9   pH                    1599 non-null   float64
 10  sulphates             1599 non-null   float64
 11  alcohol               1599 non-null   float64
 12  quality               1599 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 162.5 KB


In [136]:
# Создаём целевую переменную: 1 - хорошее вино, 0 - плохое вино
df['quality_label'] = (df['quality'] >= 6).astype(int)

# Определяем признаки и целевую переменную
X = df.drop(['Unnamed: 0', 'quality', 'quality_label'], axis=1)
y = df['quality_label']

# Разделяем данные на обучающую и тестовую выборки (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Нормализация признаков для логистической регрессии
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


2. Обучение моделей:
Логистическая регрессия

In [137]:
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)
y_pred_log_reg = log_reg.predict(X_test_scaled)
f1_log_reg = f1_score(y_test, y_pred_log_reg)
display(f1_log_reg)

0.7490196078431373

Дерево решений

In [138]:
tree_clf = DecisionTreeClassifier(random_state=42, max_depth=10)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
f1_tree = f1_score(y_test, y_pred_tree)
display(f1_tree)


0.7549407114624506

3. Определение лучшей модели:

In [139]:
best_f1_score = max(f1_log_reg, f1_tree)
print(f"Лучшее значение F1-score: {best_f1_score:.3f}")


Лучшее значение F1-score: 0.755


Обучите модель с использованием бэггинга (класс BaggingClassifier с random_state=42). Возьмите из предыдущего задания алгоритм, показавший наилучшее качество, и укажите для него новое количество моделей — 1500. Вычислите новое значение F1-score.

In [140]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import f1_score

2. Определяем базовый классификатор

In [141]:
# Используем лучший алгоритм (дерево решений)
best_model = DecisionTreeClassifier(random_state=42, max_depth=10)


3. Создаём и обучаем BaggingClassifier

In [142]:
# Создаём бэггинг с 1500 классификаторами
bagging_clf = BaggingClassifier(estimator=best_model, n_estimators=1500, random_state=42, n_jobs=-1)

# Обучаем модель
bagging_clf.fit(X_train, y_train)


BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=10,
                                                   random_state=42),
                  n_estimators=1500, n_jobs=-1, random_state=42)

4. Предсказания и расчет F1-score


In [143]:
# Получаем предсказания
y_pred_bagging = bagging_clf.predict(X_test)

# Вычисляем F1-score
f1_bagging = f1_score(y_test, y_pred_bagging)
print(f"Новое значение F1-score: {f1_bagging:.3f}")


Новое значение F1-score: 0.797


Более подробно рассматривать практическую сторону реализации случайного леса и варьировать различные параметры для получения наилучшего результата мы будем в следующем юните. Но давайте решим небольшую задачу уже сейчас. Все функции из неё вы уже использовали ранее, так что это отличный повод вспомнить их или поработать с документацией.

Мы будем анализировать набор данных Boston Houses, в котором объектами являются районы города, признаками — некие социальные и географические характеристики района, а целевой переменной — медианная стоимость домов в районе. Таким образом, мы будем решать задачу регрессии.

Как и в предыдущем практическом задании, весь код, который мы напишем, можно применить к любым данным.

crim_rate — уровень преступности в районе;

zn — доля участков площадью более 25 000 кв. футов;

business — уровень развитости бизнеса в районе;

river — наличие реки в районе;

nit_oxiden — концентрация оксидов азота в воздухе;

rooms — среднее число комнат в домах района;

age — процент домов, построенных до 1940 года;

dist — расстояние до центров занятости;

highways_index — индекс доступности крупных дорог;

tax — средняя ставка налога на имущество;

pup_per_teac — среднее число учеников на одного учителя;

lower — процент малообеспеченного населения в районе;

target — медианная стоимость домов в районе (целевая переменная).

In [144]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [145]:
# Загружаем данные
boston = pd.read_csv('data/boston (1).csv', decimal=',')
boston.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [146]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   crim_rate       506 non-null    float64
 1   zn              506 non-null    float64
 2   business        506 non-null    float64
 3   river           506 non-null    int64  
 4   nit_oxiden      506 non-null    float64
 5   rooms           506 non-null    float64
 6   age             506 non-null    float64
 7   dist            506 non-null    float64
 8   highways_index  506 non-null    int64  
 9   tax             506 non-null    int64  
 10  pup_per_teaс    506 non-null    float64
 11  lower           506 non-null    float64
 12  target          506 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 51.5 KB


1. Разбейте набор данных на обучающую и тестовую выборку в соотношении 70/30, при разбиении задайте параметр random_state = 13.

Какое получилось среднее значение медианных цен на обучающей выборке? Ответ округлите до двух знаков после точки-разделителя.

In [147]:
# Определяем признаки и целевую переменную
X = boston.drop(['target'], axis=1)
y = boston['target']

# Разделяем данные на обучающую и тестовую выборки (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

In [148]:
mean_value = sum(y_train) / len(y_train)
print(f"Среднее значение: {mean_value:.2f}")


Среднее значение: 22.77



2. Обучите линейную регрессию с параметрами по умолчанию. В качестве ответа введите ошибку MAE на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [149]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


In [150]:
# Создаём модель линейной регрессии с параметрами по умолчанию
lin_reg = LinearRegression()

# Обучаем модель на тренировочных данных
lin_reg.fit(X_train, y_train)


LinearRegression()

3. Предсказания для тестовой выборки

In [151]:
y_pred = lin_reg.predict(X_test)


4. Вычисление MAE

In [152]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE на тестовой выборке: {mae:.2f}")

MAE на тестовой выборке: 3.72


3. Обучите решающее дерево с параметрами по умолчанию и аргументом random_state = 13.

Можно ли, опираясь на результаты, сделать вывод, что алгоритм переобучился?

1. Обучение модели решающего дерева

In [153]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# Создаём модель с random_state=13
tree_reg = DecisionTreeRegressor(random_state=13)

# Обучаем модель на тренировочной выборке
tree_reg.fit(X_train, y_train)



DecisionTreeRegressor(random_state=13)

2. Оценка модели

In [154]:
# Предсказания для обучающей и тестовой выборок
y_train_pred = tree_reg.predict(X_train)
y_test_pred = tree_reg.predict(X_test)

# Вычисляем MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f"MAE на обучающей выборке: {mae_train:.2f}")
print(f"MAE на тестовой выборке: {mae_test:.2f}")


MAE на обучающей выборке: 0.00
MAE на тестовой выборке: 2.84


4. Обучите четыре случайных леса с числом деревьев 3, 10, 100, 500 и параметром random_state = 13.

В качестве ответа введите наименьшую полученную ошибку MAE на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

2. Создание и обучение моделей

In [156]:
# Задаём количество деревьев
n_trees = [3, 10, 100, 500]
mae_results = {}

# Обучаем модели и вычисляем MAE
for n in n_trees:
    forest = RandomForestRegressor(n_estimators=n, random_state=13, n_jobs=-1)
    forest.fit(X_train, y_train)
    
    # Предсказания на тестовой выборке
    y_pred = forest.predict(X_test)
    
    # Вычисление MAE
    mae_results[n] = mean_absolute_error(y_test, y_pred)

# Определяем наименьшую ошибку
best_mae = min(mae_results.values())
print(f"Наименьшая ошибка MAE: {best_mae:.2f}")


Наименьшая ошибка MAE: 2.24
